In [ ]:
#import required modules
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [ ]:
url_list = "http://www.genome.jp/dbget-bin/www_bfind_sub?dbkey=rpair&keywords=rp&mode=bfind&max_hit=nolimit"
url_pre = "http://www.genome.jp" # 이후 href와 결합하여 접근한다.

#scrap the HTML at the url
r = requests.get(url_list)

#Turn the HTML into a Beautiful Soup object
soup = BeautifulSoup(r.text, "lxml")

# get the next page link
links = soup.find_all("a")
href=[]
for link in links:
    href.append(link.get("href"))

del href[0]
del href[len(href)-1]# 맨 앞과 맨 뒤는 필요 없으므로 삭제

#각 entry의 link 주소 만들기
url = []
for i in href:
    url.append(url_pre+i)


In [ ]:
#Entry와 Name 얻기
rows = []
for row in soup.find_all("div")[1:]:
    rows.append(row.text)
entry_name = rows[::2] # entry_name에 들어가는 data 형태는 RP00001 C00005_C00006(entry와 name순)

entries=[]
names=[]
for i in entry_name:
    temp = i.split()
    entries.append(temp[0])
    names.append(temp[1])
    
#entries, names를 DataFrame으로 전환
df_entries = pd.DataFrame(data = entries, columns = ['Entry'])
df_names = pd.DataFrame(data = names, columns = ['Name'])


In [ ]:
#각 entry 페이지 불러오기    
def next_html(url):
    try:
        r2 = requests.get(url)
        #sleep(5)
    except requests.exception.ConnectionError:
        r.status_code = 'Connection refused'
    soup2 = BeautifulSoup(r2.text, "lxml")
    return soup2
    
#soup2 = next_html(url[0])
#print soup2.prettify()

In [ ]:
count = 0

In [ ]:
#파싱하기
for i in url:
    count = count +1
    soup2 = next_html(i)
    parse_html(soup2)
    if count % 100 == 0:
        print count

In [ ]:
# dataframe으로 전환
df_c1 = pd.DataFrame(data = c1, columns = ['Compound1'])
df_c1_names = pd.DataFrame(data = c1_names, columns = ['C1name'])
df_c2 = pd.DataFrame(data = c2, columns = ['Compound2'])
df_c2_names = pd.DataFrame(data = c2_names, columns = ['C2name'])
df_types = pd.DataFrame(data = types, columns = ['Type'])
df_rpairs = pd.DataFrame(data = rpairs, columns = ['Related piar'])

In [ ]:
result = pd.DataFrame()
result['Entry'] = df_entries
result['Name'] = df_names
result['Compound1'] = df_c1
result['C1name'] = df_c1_names
result['Compound2'] = df_c2
result['C2name'] = df_c2_names
result['Type'] = df_types
result['Related rpair'] = df_rpairs

In [ ]:
result = pd.DataFrame()

In [ ]:
result.to_csv("result", mode='w', encoding='utf-8', sep='\t')
df_entries.to_csv("df_entries", mode='w', encoding='utf-8', sep='\t')
df_names.to_csv("df_names", mode='w', encoding='utf-8', sep='\t')
df_c1.to_csv("df_c1", mode='w', encoding='utf-8', sep='\t')
df_c1_names.to_csv("df_c1_names", mode='w', encoding='utf-8', sep='\t')
df_c2.to_csv("df_c2", mode='w', encoding='utf-8', sep='\t')
df_c2_names.to_csv("df_c2_names", mode='w', encoding='utf-8', sep='\t')
df_types.to_csv("df_types", mode='w', encoding='utf-8', sep='\t')
df_rpairs.to_csv("df_rpairs", mode='w', encoding='utf-8', sep='\t')


In [ ]:
result

In [ ]:
c1 = df_c1.values.tolist() # c1에 문제가 생겨서 복구한것

In [ ]:
# 현재 c1, c2, c1_names, c2_names, types, rpairs의 인덱스 기준 0~3282까지 중복이 일어난 상태
#중복을 제거하고 본래 데이터를 찾기

#기존 값을 보존하기 위해 새로운 list를 만든다.
new_c1 = c1[:]
new_c2 = c2[:] 
new_c1_names = c1_names[:]
new_c2_names = c2_names[:]
new_types = types[:]
new_rpairs = rpairs[:]


#인덱스 0부터 3282를 삭제한다
new_c1[0:3283] = []
new_c2[0:3283] = []
new_c1_names[0:3283] = []
new_c2_names[0:3283] = []
new_types[0:3283] = []
new_rpairs[0:3283] = []



#unicode라서 ascii로 바꿀 필요가 있다.
str_c1=[]
str_c1_names=[]
str_c2=[]
str_c2_names=[]
str_types=[]
str_rpairs=[]

#ascii로 바꾸면서 붙는 non-ascii character를 제거
for i in range(0, len(new_c1)):
    str_c1.append(new_c1[i][0].encode('ascii', 'ignore'))
    str_c1_names.append(new_c1_names[i].encode('ascii', 'ignore'))
    str_c2.append(new_c2[i].encode('ascii', 'ignore'))
    str_c2_names.append(new_c2_names[i].encode('ascii', 'ignore'))
    str_types.append(new_types[i].encode('ascii', 'ignore'))
    str_rpairs.append(new_rpairs[i].encode('ascii', 'ignore'))



#수정된 데이터로 dataframe을 만든다.
df_str_c1 = pd.DataFrame(data = str_c1, columns = ['Compound1'])
df_str_c1_names = pd.DataFrame(data = str_c1_names, columns = ['C1name'])
df_str_c2 = pd.DataFrame(data = str_c2, columns = ['Compound2'])
df_str_c2_names = pd.DataFrame(data = str_c2_names, columns = ['C2name'])
df_str_types = pd.DataFrame(data = str_types, columns = ['Type'])
df_str_rpairs = pd.DataFrame(data = str_rpairs, columns = ['Related piar'])




In [ ]:
#수정된 데이터로 result matrix를 만든다.
str_result = pd.DataFrame()
str_result['Entry'] = df_entries
str_result['Name'] = df_names
str_result['Compound1'] = df_str_c1
str_result['C1name'] = df_str_c1_names
str_result['Compound2'] = df_str_c2
str_result['C2name'] = df_str_c2_names
str_result['Type'] = df_str_types
str_result['Related rpair'] = df_str_rpairs



In [ ]:
#수정된 데이터로 만들어진 result matrix를 파일로 저장한다. 

str_result.to_csv("str_result.csv", mode='w', index=False)
df_entries.to_csv("str_df_entries.csv", mode='w', index=False)
df_names.to_csv("str_df_names.csv", mode='w', index=False)
df_str_c1.to_csv("str_df_c1.csv", mode='w', index=False)
df_str_c1_names.to_csv("str_df_c1_names.csv", mode='w', index=False)
df_str_c2.to_csv("str_df_c2.csv", mode='w', index=False)
df_str_c2_names.to_csv("str_df_c2_names.csv", mode='w', index=False)
df_str_types.to_csv("str_df_types.csv", mode='w', index=False)
df_str_rpairs.to_csv("str_df_rpairs.csv", mode='w', index=False)


In [ ]:
str_result

In [ ]:
print new_c1[0][0].encode('ascii', 'replace')
print new_c1[0][0].encode('ascii', 'ignore')

a = new_c1[0][0].encode('ascii', 'ignore')
type(a)

In [ ]:
str_result.to_csv("2.csv", mode='w', index=False)